In [ ]:
#             Notebook #1
#                   Chop Therapy Session transcript files and generate some Features
#
#              
# This notebook reads all the transcripts in '/Users/Heisig/Jihan/Transcripts/' 
# It creates split subject interviewer transcripts here: /Users/Heisig/Jihan/Transcripts/SplitTranscripts/'
# It creates split subject interviewer turn transcripts here: /Users/Heisig/Jihan/Transcripts/SplitTurnsTranscripts'
# It creates Google USE embedding vectors and saves them in: '/Users/Heisig/Jihan/Embeddings/'
# It creates POS tagged tokens using stanza and saves them in: '/Users/Heisig/Jihan/POS/'
# A few plots and stats are available
# Then go run LIWC on the split files
# Then run the LIWC per Turn Notebook

#  Caveat Lector: The notebook is structured for our file system structure and input data types. It will
#                 require some refactoring to run on other structures.

In [ ]:

import pandas as pd
import stanza 
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from docx.api import Document

import nltk
from nltk import ngrams
from nltk.lm import MLE
import contractions
from collections import Counter
import string
from string import punctuation

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))



#      Options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from IPython.core.display import display, HTML

from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseQuotes": False}}})

def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

#Populate the turn DF 
def create_turnDF_old(all_turns):
    speakerDict = {'T':'therapist','P':'tatient'}
    turnDF = pd.DataFrame(columns=['Turn','Speaker','Text'])
    turn = 0
    for i in range(len(all_turns)-1):
        currentParts = all_turns[i].text.split(":")
        if len(currentParts)==2:
            print(all_turns[i].text)
            speaker = speakerDict[currentParts[0].lstrip()]
            text = currentParts[1]
            [clean_list,non_verbals] = cleanTurn(text)
            print('clean_list: ',clean_list)
            print('non_verbals: ',non_verbals)        
            #Turn back into a string
            clean_text = ' ' .join(clean_list) 
            newRow = [turn,speaker,clean_text]
            turnDF.loc[turn] = newRow
            turn = turn + 1
    return turnDF

#Populate the turnDF and posDF
def create_turnDF(all_turns):
    #turnDF = pd.DataFrame(columns=['turn','sentence','text'])
    speakerDict = {'T':'therapist','P':'patient'}
    turnDF = pd.DataFrame(columns=['Turn','Sentence','Speaker','Text'])
    turn = 0
    posDF = pd.DataFrame(columns=['subjectsession','speaker','sentence','text','lemma','upos','xpos','deprel'])
    sentenceTotalNum = 0
    for i in range(len(all_turns)):
        currentParts = all_turns[i].text.split(":")
        if len(currentParts)==2:
            print(all_turns[i].text)
            
            #Original version 11/18/21
            #speaker = currentParts[0].lstrip()
            
            #HACK Test 11/18/21???
            speaker = speakerDict[currentParts[0].lstrip()]
            
            
            text = currentParts[1]

            turn_text = text
            
            #                         Sentencize and Parse with stanza
            #Note once you get the text into a Stanza doc you can do all sorts of interesting things
            doc = nlp(turn_text)

            sentenceNum = 0
            
            for sentence in doc.sentences:
               turnRow = [turn,sentenceTotalNum,speaker,sentence.text]
               turnDF.loc[len(turnDF)+1] = turnRow
               
               #Populate the POS 
               for word in sentence.words:
                   posDF.loc[len(posDF)+1] = [SubjectSession,speaker,sentenceTotalNum, word.text, word.lemma, word.upos, word.xpos, word.deprel]
               sentenceNum = sentenceNum + 1
               sentenceTotalNum = sentenceTotalNum + 1
               
        turn = turn + 1
    turnDF.reset_index(inplace=True,drop=True) 
    print('type(turnDF): ',type(turnDF))
    return [turnDF, posDF]

#Create the embedding dataframe and write out the patient/therapist text
def create_embeddingDF(turnDF):
    #Create the dataframe
    columnNames = ['subjectsession','speaker','turn','sentence','wordCount','charCount','charsPerWord','text']
    columnNames.extend(range(512))
    embeddingDF = pd.DataFrame(columns=columnNames)
    
    embeddings = embed(turnDF['Text'])

    for i, message_embedding in enumerate(np.array(embeddings).tolist()):
        turnText = turnDF.at[i,'Text']
        print('turnText: ',turnText)
        speaker = turnDF.at[i,'Speaker']
        sentence = turnDF.at[i,'Sentence']
        turn = turnDF.at[i,'Turn']
        wordCount = len(turnText.split()) 
        if wordCount==0:
            continue
        charCount = len(turnText)
        charsPerWord = charCount*1.0/wordCount*1.0
        newRow = [SubjectSession,speaker,turn,sentence,wordCount,charCount,charsPerWord,turnText]
        newRow.extend(message_embedding)
        #print(newRow)
        embeddingDF.loc[len(embeddingDF)] = newRow
        
    embeddingFileName = '/Users/Heisig/Jihan/Embeddings/'+SubjectSession+"_Embedding.csv"

    #embeddingDF.drop('subjectsession', axis=1, inplace=True)
    #embeddingDF.drop('turn', axis=1, inplace=True)

    embeddingDF.to_csv(embeddingFileName, index=False, header=True, sep=',')

    #Now save seperate patient/therapist text files for LIWC processing
    patientFileName = '/Users/Heisig/Jihan/Transcripts/SplitTranscripts/'+SubjectSession+"_patient.txt"
    patientDF = embeddingDF.loc[embeddingDF.speaker=='Subject','text']
    patientDF.to_csv(patientFileName, index=False, header=False, sep=',')

    therapistFileName = '/Users/Heisig/Jihan/Transcripts/SplitTranscripts/'+SubjectSession+"_therapist.txt"
    therapistDF = embeddingDF.loc[embeddingDF.speaker=='Interviewer','text']
    therapistDF.to_csv(therapistFileName, index=False, header=False, sep=',')
    
    return embeddingDF

def run_and_plot(session_, input_tensor_, messages_, encoding_tensor):
  message_embeddings_ = session_.run(
      encoding_tensor, feed_dict={input_tensor_: messages_})
  plot_similarity(messages_, message_embeddings_, 90)
    
def cleanTurn(turn):
    obfuscated_words = ['xxx','XXX']
    
    #Save non-verbals
    regex = re.compile(".*?\((.*?)\)")
    non_verbals = re.findall(regex, turn)
    
    #Remove non-verbals
    turn = re.sub(r'\([^)]*\)', '', turn)
   
    #Expand contractions
    #turn = contractions.fix(turn)
    
    #Get rid of digits
    turn = ''.join([i for i in turn if not i.isdigit()])
    
    #Get rid of punctuation EXCEPT for periods
    #In Jihan's data there actually aren't any periods as sentence delimiters
    #There are some elipses
    #So using sentence boundaries doesn't make sense
    
    #Get rid of punctuation EXCEPT for periods
    #punctuation_set = punctuation.replace(".", "")
    #turn = turn.translate(str.maketrans("", "", punctuation_set))
    
    #Get rid of ALL punctuation
    #turn = turn.translate(str.maketrans('', '', string.punctuation))
    
    #Get rid of punctuation EXCEPT for periods, question marks, and exclamation points.
    punctuation_set = punctuation.replace(".", "")
    punctuation_set = punctuation_set.replace("?", "")
    punctuation_set = punctuation_set.replace("!", "")
    turn = turn.translate(str.maketrans("", "", punctuation_set))
    
    #Switch to lower case
    #turn = turn.lower()
    
    clean_list = turn.split() 
    
    #Get rid of obfuscated words
    clean_list = [word for word in clean_list if not word in obfuscated_words]
    
    return [clean_list,non_verbals]

def cleanSentence(sentence):
    obfuscated_words = ['XXX','xxx','ykwis','ykis','ykwim',]
    
    #Save non-verbals
    regex = re.compile(".*?\((.*?)\)")
    non_verbals = re.findall(regex, sentence)
    
    #Remove non-verbals
    sentence = re.sub(r'\([^)]*\)', '', sentence)
   
    #Expand contractions
    #sentence = contractions.fix(sentence)
    
    #Get rid of digits
    sentence = ''.join([i for i in sentence if not i.isdigit()])

    #Get rid of punctuation EXCEPT for periods, question marks, and exclamation points.
    punctuation_set = punctuation.replace(".", "")
    punctuation_set = punctuation_set.replace("?", "")
    punctuation_set = punctuation_set.replace("!", "")
    turn = turn.translate(str.maketrans("", "", punctuation_set))
    
    #Switch to lower case
    #sentence = sentence.lower()
    
    clean_list = sentence.split() 
    
    #Get rid of obfuscated words
    clean_list = [word for word in clean_list if not word in obfuscated_words]
    
    clean_sentence = ' '.join(clean_list)
    
    return clean_sentence


def ngramStats(allTextList,n):
    #Compute word frequencies
    word_fdist = nltk.FreqDist(allTextList)    
    wordDF = pd.DataFrame.from_dict(word_fdist, orient='index')
    wordDF['word'] = wordDF.index
    wordDF.columns = ['count','word']  
    wordDF = wordDF[['word','count']] 
    wordDF.sort_values(by=['count'], ascending=False, inplace=True)
    wordDF.reset_index(inplace=True,drop=True)
    wordDF.head()

    #Compute n gram frequencies
    ngram_fd = nltk.FreqDist(ngrams(allTextList,n))
    ngramDF = pd.DataFrame(ngram_fd.most_common(), columns =['w1-wn', 'Freq']) 
    return[wordDF,ngramDF]
    

In [ ]:
#       Pull in the model
#Off world
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#Local
embed = hub.load('/Users/Heisig/Jihan/USE/universal-sentence-encoder_4') 
stanza.download('en') # download English model
#Initialize a Stanza Pipeline
nlp = stanza.Pipeline(lang='en', processors='tokenize,lemma,pos,depparse,ner')

In [ ]:
# Read all the split transcripts and generate per turn files for LIWC
# Generate allTurnsTextDF

documentDF = pd.DataFrame(columns=['SubjectSession','document'])
                          
InputDocumentDir = '/Users/Heisig/Jihan/Transcripts/'
turnsDir = '/Users/Heisig/Jihan/Transcripts/SplitTurnsTranscripts/'
                          
file_paths = []  # List which will store all of the full filepaths.
# Walk the directory tree.
for root, directories, files in os.walk(InputDocumentDir):
    for filename in files:
        # Join the two strings in order to form the full filepath.
        filepath = os.path.join(root, filename)
        #print('root: ',root)
        #print('filename: ',filename)
        file_paths.append(filepath)  # Add it to the list.
        
docxFiles  = [s for s in file_paths if '.docx' in s]
print('docxFiles: \n',docxFiles)

allTurnsTextDF = pd.DataFrame(columns=['SubjectSession', 'Speaker', 'Turn', 'Text'])

for docxFile in docxFiles:   
    document = Document(docxFile)
    all_turns = document.paragraphs
    path_file = os.path.split(docxFile) 
    fileName = path_file[1]
    fileParts = fileName.split('.')
    SubjectSession = fileParts[0]

    #Create the turns dataframe
    [turnDF, posDF] = create_turnDF(all_turns)
    turnDF.insert(0, 'SubjectSession', [SubjectSession]*len(turnDF))
    allTurnsTextDF = allTurnsTextDF.append(turnDF, ignore_index=True)
    
    patientTurnsDF = turnDF.loc[turnDF['Speaker']=='Subject']
    patientTurnsDF.reset_index(inplace=True,drop=True) 
    patientTurns, cols = patientTurnsDF.shape
    interviewerTurnsDF = turnDF.loc[turnDF['Speaker']=='Interviewer']
    interviewerTurnsDF.reset_index(inplace=True,drop=True)
    interviewerTurns, cols = interviewerTurnsDF.shape
    totalTurns = patientTurns + interviewerTurns
    
    #Write each turn to its own file so LIWC can pick it up individually
    sessionDir = turnsDir+SubjectSession+'/'
    if os.path.isdir(sessionDir):
       print(sessionDir + ' already exists')
    else: 
       os.mkdir(sessionDir)
       print(sessionDir + ' created') 
     
    for index, row in patientTurnsDF.iterrows():
        outFileName = sessionDir+SubjectSession+'_patient_'+str(totalTurns)+'_'+str(row['Sentence'])+'.txt'
        outF = open(outFileName, 'w')
        outF.write(row['Text'])
        outF.close()    
    for index, row in interviewerTurnsDF.iterrows():
        outFileName = sessionDir+SubjectSession+'_interviewer_'+str(totalTurns)+'_'+str(row['Sentence'])+'.txt'
        outF = open(outFileName, 'w')
        outF.write(row['Text'])
        outF.close()  

#Fixup for merge with LIWC and Clinical later
speakerDict = {'Interviewer' : 'interviewer', 'Subject' : 'patient'} 
allTurnsTextDF.replace({"Speaker": speakerDict},inplace=True)
allTurnsTextDF['SubjectSession'] = allTurnsTextDF['SubjectSession'].astype('int')
allTurnsTextDF['SubjectSession'] = allTurnsTextDF['SubjectSession'].astype('str')


#Save the turn text file
turnTextFileName = '/Users/Heisig/Jihan/LIWC/TurnText.csv'
allTurnsTextDF.to_csv(turnTextFileName)


In [ ]:
#                 Process all the transcripts New Style (whole turn)


# Load the first table from your document. In your example file,
# there is only one table, so I just grab the first one.


documentDF = pd.DataFrame(columns=['SubjectSession','document'])
                          
InputDocumentDir = '/Users/Heisig/Jihan/Transcripts/'
StatsDir = '/Users/Heisig/Jihan/Stats/'
                          
file_paths = []  # List which will store all of the full filepaths.
# Walk the directory tree.
for root, directories, files in os.walk(InputDocumentDir):
    for filename in files:
        # Join the two strings in order to form the full filepath.
        filepath = os.path.join(root, filename)
        #print('root: ',root)
        #print('filename: ',filename)
        file_paths.append(filepath)  # Add it to the list.
        
docxFiles = sorted([s for s in file_paths if ".docx" in s])

allAllPatientList = []
allAllTherapistList = []   
 
    
for docxFile in docxFiles:
    document = Document(docxFile)
    all_turns = document.paragraphs
    path_file = os.path.split(docxFile) 
    fileName = path_file[1]
    fileParts = fileName.split('.')
    SubjectSession = fileParts[0]
    
    #Create the turnDF 
    print('SubjectSession: ',SubjectSession)
    [turnDF,posDF] = create_turnDF(all_turns)
    
    speakerDict = {'Interviewer' : 'therapist', 'Subject' : 'patient'} 
    allTurnsTextDF.replace({"Speaker": speakerDict},inplace=True)
    
    #Save the POS files
    posFileName = '/Users/Heisig/Jihan/POS/'+SubjectSession+"_POS.csv"
    posDF.to_csv(posFileName, index=False, header=True, sep=',')

    
    #Create (and write out) the embeddings for each turn
    embeddingDF = create_embeddingDF(turnDF)
        

In [ ]:
patientEmbeddingDF = embeddingDF.loc[embeddingDF['speaker']=='patient']
therapistEmbeddingDF = embeddingDF[embeddingDF['speaker']=='therapist']

fig, ax = plt.subplots()
fig.set_size_inches(16, 8)


plt.hist([patientEmbeddingDF[0], therapistEmbeddingDF[0]], 
         bins=100,
         color=['r','b'], 
         label=['patient','therapist'],
         alpha=0.5)

plt.legend()

#sns.distplot(vector, label=row,bins=250, ax=ax, hue='speaker')
#plotTitle = 'embedding vector '+str(row)
#sns.distplot(vector, label=row,bins=250,ax=ax).set_title(plotTitle)
#plt.legend();
#ax.set(xlabel='PreOp '+metric, ylabel='Observations')
#ax.set_xlim(0,1)
  
#plotFileNPree = outputDir+Subject+"_"+metric+"_Histo.png"
#fig.savefig(plotFileNPree)
#plt.close(fig)


In [ ]:
allPatientStr = patientEmbeddingDF['text'].str.cat(sep=' ')
print('allPatientStr: \n',allPatientStr)
allPatientList = allPatientStr.split()
n = 4
wordDF,ngramDF = ngramStats(allPatientList,n)
print('ngramDF: \n',ngramDF.head(n=25))

In [ ]:
allTherapistStr = therapistEmbeddingDF['text'].str.cat(sep=' ')
print('allTherapistStr : \n',allTherapistStr )
allTherapistList = allTherapistStr.split()
n = 4
wordDF,ngramDF = ngramStats(allTherapistList,n)
print('ngramDF: \n',ngramDF.head(n=25))

In [ ]:
#Count therapist words
print('Total therapist words: ',len(allTherapistList)) 
therapist_word_fdist = nltk.FreqDist(allTherapistList) 
therapist_fdistDF = pd.DataFrame.from_dict(therapist_word_fdist, orient='index')
therapist_fdistDF['word'] = therapist_fdistDF.index
therapist_fdistDF.columns = ['therapist_count','word'] 
therapist_fdistDF['therapist_freq'] = therapist_fdistDF['therapist_count']/len(allTherapistList)
therapist_fdistDF.sort_values(by=['therapist_count'], ascending=False, inplace=True)
therapist_fdistDF.reset_index(inplace=True,drop=True)
therapist_fdistDF.head(n=25)

In [ ]:
#Count patient words
print('Total patient words: ',len(allPatientList)) 
patient_word_fdist = nltk.FreqDist(allPatientList) 
patient_fdistDF = pd.DataFrame.from_dict(patient_word_fdist, orient='index')
patient_fdistDF['word'] = patient_fdistDF.index
patient_fdistDF.columns = ['patient_count','word'] 
patient_fdistDF['patient_freq'] = patient_fdistDF['patient_count']/len(allPatientList)
patient_fdistDF.sort_values(by=['patient_count'], ascending=False, inplace=True)
patient_fdistDF.reset_index(inplace=True,drop=True)
patient_fdistDF.head(n=25)

In [ ]:
total_fdistDF = pd.merge(patient_fdistDF,therapist_fdistDF, on='word')
total_fdistDF.head(n=100)

In [ ]:
#Plot Patient and Therapist Freq Word freq
fig = plt.gcf()
fig.set_size_inches(12, 8)
plt.gcf().subplots_adjust(bottom=0.15)

ax = sns.scatterplot(data=total_fdistDF,
                     x="patient_freq",
                     y="therapist_freq");
ax.set_xlim(0,0.09)
ax.set_ylim(0,0.09)

for lineNum in range(0,25):
             pointLabel = total_fdistDF.word[lineNum]
             ax.text(total_fdistDF.loc[lineNum,'patient_freq'], total_fdistDF.loc[lineNum,'therapist_freq'], pointLabel, horizontalalignment='left', size='small', alpha=0.99, color='black', weight='roman')


In [ ]:
#List of all .docx files 
allDocxFiles = [s for s in file_paths if '.docx' in s]
print(allDocxFiles)

In [ ]:
#                  Old Style Comma and Period Parser

#Single fle HACK
allDocxFiles = ['/Users/Heisig/Jihan/Transcripts/030920.docx']

for docFile in allDocxFiles:

    path_file = os.path.split(docxFile) 
    fileName = path_file[1]
    fileParts = fileName.split('.')
    SubjectSession = fileParts[0]
    
    
    document = Document(docFile)
    all_turns = document.paragraphs
    parts = docFile.split('/')
    print(parts[5])
    path_file = os.path.split(docFile) 
    fileName = path_file[1]
    fileParts = fileName.split('.')
    SubjectSession = fileParts[0]

    #Populate the turn DF 
    speakerDict = {'T':'Interviewer','P':'Subject'}
    turnDF = pd.DataFrame(columns=['Turn','Speaker','Text'])
    turn = 0
    for i in range(len(all_turns)-1):
        currentParts = all_turns[i].text.split(":")
        if len(currentParts)==2:
            #print(all_turns[i].text)
            speaker = speakerDict[currentParts[0]]
            text = currentParts[1]
            newRow = [turn,speaker,text]
            turnDF.loc[turn] = newRow
            turn = turn + 1
    sentenceList = []
    sentenceSourceList = []
    parseType = 'Commas'
    #parseType = 'Sentences'
    for index, row in turnDF.iterrows():
        turn = row['Text']
        start = turn.find( '(' )
        end = turn.find( ')' )
        if start != -1 and end != -1:
           turn = turn[start+1:end]
        turn = turn.replace('(','')
        turn = turn.replace(')','')
        if parseType == 'Commas':
            turn = turn.replace(',','.')
        else:
            turn = turn.replace(',','')
        turn = turn.replace('?','.')
        turn = turn.replace('!','.')
        #print('turn: ',turn)
        sentences = turn.split('.')
        #print('sentences: ',sentences)
        while("" in sentences): 
            sentences.remove("")
        for sentence in sentences:
            sentence = sentence.strip()
            sentence = cleanSentence(sentence)
            if len(sentence)>1:
                sentenceList.append(sentence)
                sentenceSourceList.append(row['Speaker'])
                print(sentence)
        
    #Create the dataframe
    columnNames = ['subjectsession','speaker','turn','text','wordCount','charCount','charsPerWord']
    columnNames.extend(range(512))
    print('len(columnNames): ',len(columnNames))
    embeddingDF = pd.DataFrame(columns=columnNames)

    embeddings = embed(sentenceList)

    for i, message_embedding in enumerate(np.array(embeddings).tolist()):
        #print(message_embedding)
        wordCount = len(sentenceList[i].split()) 
        charCount = len(sentenceList[i])
        charsPerWord = charCount*1.0/wordCount*1.0
        newRow = [SubjectSession,sentenceSourceList[i],i,sentenceList[i],wordCount,charCount,charsPerWord]
        newRow.extend(message_embedding)
        #print(newRow)
        #print(len(newRow),len(newRow))
        embeddingDF.loc[len(embeddingDF)] = newRow
    embeddingFileName = '/Users/Heisig/Jihan/OldStyleEmbeddings/'+SubjectSession+"_"+parseType+"_Embedding.csv"

    embeddingDF.drop('subjectsession', axis=1, inplace=True)
    #embeddingDF.drop('turn', axis=1, inplace=True)

    embeddingDF.to_csv(embeddingFileName, index=False, header=True, sep=',')
    """
    #Now save seperate patient/therapist text files for LIWC processing
    patientFileName = '/Users/Heisig/Jihan/Transcripts/'+SubjectSession+"_patient.txt"
    patientDF = embeddingDF.loc[embeddingDF.speaker=='Subject','text']
    patientDF.to_csv(patientFileName, index=False, header=True, sep=',')

    therapistFileName = '/Users/Heisig/Jihan/Transcripts/'+SubjectSession+"_therapist.txt"
    therapistDF = embeddingDF.loc[embeddingDF.speaker=='Interviewer','text']
    therapistDF.to_csv(therapistFileName, index=False, header=True, sep=',')
    """

In [ ]:
word_fdist = nltk.FreqDist(sentence_list) 
fdistDF = pd.DataFrame.from_dict(word_fdist, orient='index')

In [ ]:
allAllTherapistList

In [ ]:
patientEmbeddingDF

In [ ]:
#Read a previously unseen session
docxFile = '/Users/Heisig/Jihan/Validation_Transcript/111920.docx'
print('docxFile: ',docxFile)
document = Document(docxFile)
path_file = os.path.split(docxFile) 
fileName = path_file[1]
fileParts = fileName.split('.')
SubjectSession = fileParts[0]

all_turns = document.paragraphs

#This returns 'cleaned' turn text for each speaker
#Commas really aren't a legit way to split the text further
#So we will either use the whole turn or ngram our way through
#it.
validity_turnDF = create_turnDF(all_turns)
validity_therapistDF = embeddingDF.loc[embeddingDF.speaker=='Interviewer',:]
validity_patientDF = embeddingDF.loc[embeddingDF.speaker=='Subject',:]
print(validity_patientDF.columns)

In [ ]:
# Unigram Train/Test Perplexity Evaluation

import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

# Jihan Patient, Therapist Training data
tokenized_patient_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in patientEmbeddingDF['text']]

tokenized_therapist_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in therapistEmbeddingDF['text']]

tokenized_patient_validity_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in validity_patientDF['text']]

tokenized_therapist_validity_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in validity_therapistDF['text']]

#Train unigram model on patient speech
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_patient_text)
patient_model = MLE(n)
patient_model.fit(train_data, padded_vocab)


#Train unigram model on therapist speech
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_therapist_text)
therapist_model = MLE(n)
therapist_model.fit(train_data, padded_vocab)


#Evaluate the patient validity transcript using the Patient model MLE and PP
print('\n')
print('\n')
print('Patient Validity vs. Patient Model') 
test_data, _ = padded_everygram_pipeline(n, tokenized_patient_validity_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),patient_model.score(ngram[-1], ngram[:-1])) for ngram in test])
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(print('test_data: ',test_data)[i], patient_model.perplexity(test)))

#Evaluate the patient validity transcript using Therapist model MLE and PP
print('\n')
print('\n')
print('Patient Validity vs. Patient and Therapist Models')
test_data, _ = padded_everygram_pipeline(n, tokenized_patient_validity_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),therapist_model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_patient_validity_text)
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(tokenized_patient_validity_text[i], therapist_model.perplexity(test)))


#Evaluate vs. therapist
test_data, _ = padded_everygram_pipeline(n, tokenized_therapist_validity_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),patient_model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_therapist_validity_text)
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(tokenized_therapist_validity_text[i], therapist_model.perplexity(test)))



#Evaluate Hand-whittled sentences vs. Therapist Model
print('/n')
print('/n')
print('Test Sentences vs. Therapist Model')
test_sentences = ['i am really tired', 'i do not know', 'you could think about it']
tokenized_test_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_sentences]
test_data, _ = padded_everygram_pipeline(n, tokenized_test_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),therapist_model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_test_text)
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(tokenized_test_text[i], therapist_model.perplexity(test)))


#Evaluate Hand-whittled sentences vs. Patient Model
print('/n')
print('/n')
print('Test Sentences vs. Patient Model')
test_sentences = ['i am really tired', 'i do not know', 'you could think about it']
tokenized_test_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_sentences]
test_data, _ = padded_everygram_pipeline(n, tokenized_test_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),patient_model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_test_text)
for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(tokenized_test_text[i], patient_model.perplexity(test)))

In [ ]:
tokenized_patient_validity_text

In [ ]:
#Vanilla Model

import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

train_sentences = ['an apple', 'an orange']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in train_sentences]
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_sentences = ['an apple', 'an ant']
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_sentences]

test_data, _ = padded_everygram_pipeline(n, tokenized_text)
for test in test_data:
    print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

test_data, _ = padded_everygram_pipeline(n, tokenized_text)

for i, test in enumerate(test_data):
  print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))